In [2]:
from datetime import date
date.today()

datetime.date(2017, 12, 9)

In [3]:
author = "NirantK. https://github.com/NirantK/keras-practice"
print(author)

NirantK. https://github.com/NirantK/keras-practice


In [4]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [5]:
import keras

Using TensorFlow backend.


In [6]:
keras.__version__

'2.0.8'

In [7]:
import os
if os.name=='nt':
    print('We are on Windows')

We are on Windows


Prerequisites: 
- Python programming experience
- Familiarity with machine learning vocabulary
   * Refer [Ailyen Blog](http://blog.aylien.com/10-machine-learning-terms-explained-in-simple/) - 5 minutes read or [Google's Machine Learning Glossary](https://developers.google.com/machine-learning/glossary/) when you see/hear new terms
    
Nice to have (will build upon these topics)
- Familiarity with deep feed forward networks
- Understand backpropagation 
- Familiarity with activation functions like relu, tanh, softmax, hierarchical softmax

**We will start with an introduction to the theory behind convnets, specifically:**

- What is convolution and max-pooling?   
- What are convnets?
- What do convnets learn?

Then we will cover image classification with **small datasets**:

- Training your own small convnets from scratch
- Using data augmentation to mitigate overfitting
- Using a pre-trained convnet to do feature extraction
- Fine-tuning a pre-trained convnet

We will use our convnet to classify **MNIST digits**

In [8]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

The 6 lines of code show you what a basic convnet looks like. It’s a stack of ```Conv2D``` and ```MaxPooling2D``` layers. 

**We’ll see in a minute what they do concretely**

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


The next step would be to feed our last output tensor (of shape (3, 3, 64)) into a densely-connected classifier network like those you are already familiar with: *a stack of Dense layers*. 

These classifiers process vectors, which are 1D, whereas our current output is a 3D tensor. So first, we will have to flatten our 3D outputs to 1D, and then add a few Dense layers on top:

In [12]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                36928     
__________

In [17]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=50, batch_size=64)

Epoch 1/50
60000/60000 [==============================] - 6s - loss: 0.0163 - acc: 0.9948     
Epoch 2/50
60000/60000 [==============================] - 5s - loss: 0.0128 - acc: 0.9957     
Epoch 3/50
60000/60000 [==============================] - 5s - loss: 0.0116 - acc: 0.9965     
Epoch 4/50
60000/60000 [==============================] - 5s - loss: 0.0093 - acc: 0.9974     
Epoch 5/50
60000/60000 [==============================] - 5s - loss: 0.0081 - acc: 0.9977     
Epoch 6/50
60000/60000 [==============================] - 5s - loss: 0.0067 - acc: 0.9980     
Epoch 7/50
60000/60000 [==============================] - 5s - loss: 0.0064 - acc: 0.9981     
Epoch 8/50
60000/60000 [==============================] - 5s - loss: 0.0056 - acc: 0.9983     
Epoch 9/50
60000/60000 [==============================] - 5s - loss: 0.0044 - acc: 0.9985     
Epoch 10/50
60000/60000 [==============================] - 5s - loss: 0.0045 - acc: 0.9987     
Epoch 11/50
60000/60000 [========================

KeyboardInterrupt: 

# SHOW OFF

In [18]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

 9824/10000 [============================>.] - ETA: 0s

0.9919

**Let's start by understanding the code above**
### Line 1

```python
model = model.Sequential()
```
Keras supports two different API flavours. The first is the sequential version. This works best for linear stacks of layers, which is the most common network architecture by far, and the *functional API* - for directed acyclic graphs of layers, allowing to build completely arbitrary architectures.

For the forseeable examples, we will focus on ```Sequential``` models only, but for your reference here are two code sample doing exactly the same thing in Sequential and functional API both. 

In [9]:
# Sequential Model
from keras import models
from keras import layers

model_sequential = models.Sequential()
model_sequential.add(layers.Dense(32, activation='relu', input_shape=(784,)))
model_sequential.add(layers.Dense(10, activation='softmax'))

In [10]:
# Same model as above in Functional API
input_tensor = layers.Input(shape=(784,))
x = layers.Dense(32, activation='relu')(input_tensor)
output_tensor = layers.Dense(10, activation='softmax')(x)

model_functional_api = models.Model(input=input_tensor, output=output_tensor)

C:\Users\nirantk\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


### Line 2
```python
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
```
In the model object, we are adding more objects using ```add()```. 

Here, we are adding the ```layer``` object where the object type is ```Conv2D```. We will configure our convnet to process inputs of size ```(28, 28, 1)```, which is the format of MNIST images. We do this via passing the argument ```input_shape=(28, 28, 1)``` to our first layer.

Importantly, a convnet takes as input tensors of shape ```(image_height, image_width, image_channels)``` (not including the batch dimension). 